In [13]:
import mlflow
import mlflow.sklearn
import mlflow.tensorflow
import time
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score,classification_report
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import pickle
from mlflow.models.signature import infer_signature
import warnings
warnings.filterwarnings('ignore')

In [14]:
data_dict = pickle.load(open('./data_merged.pickle', 'rb'))
data = np.asarray(data_dict['data'])
labels = np.asarray(data_dict['labels'])
labels=labels.astype(int)

In [15]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, shuffle=True, stratify=labels)

In [16]:

mlflow.tensorflow.autolog()
mlflow.sklearn.autolog()



# ---------------------------------------------------------
# Helper function to log results
# ---------------------------------------------------------
def evaluate_and_log(model_name, predict_fn):
    start = time.time()

    preds = predict_fn()
    accuracy = accuracy_score(y_test, preds)

    duration = time.time() - start
    print(f"{model_name} accuracy = {accuracy:.4f}")
    return classification_report(y_test, preds, output_dict=True)



rf_model = RandomForestClassifier(
        n_estimators=300,
        max_depth=None,
        random_state=42
    )

rf_model.fit(X_train, y_train)

evaluate_and_log(
        "RandomForest",
        lambda: rf_model.predict(X_test)
    )


model2 = models.Sequential([
       
        layers.Dense(256, activation='relu',input_shape=X_train[0].shape),
        layers.Dense(128, activation='relu'),
        layers.Dense(28, activation='softmax')
    ])

model2.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

model2.fit(X_train, y_train, epochs=5, batch_size=2048, validation_split=0.1)

preds = np.argmax(model2.predict(X_test), axis=1)
evaluate_and_log("CNN_Model_2", lambda: preds)



num_classes = 28

model3 = models.Sequential([
        layers.Dense(128, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])

model3.compile(optimizer='adam',
                   loss='sparse_categorical_crossentropy',
                   metrics=['accuracy'])

model3.fit(X_train, y_train, epochs=5, batch_size=2048, validation_split=0.1)

preds = np.argmax(model3.predict(X_test), axis=1)
evaluate_and_log("CNN_Model_3", lambda: preds)

models = [
    
    (
        "Random Forest",        # Model name              # Parameters for Random Forest
        rf_model,  # Model instance
        (X_train, y_train),     # Training data
        (X_test, y_test)        # Testing data
    ),
    (
        "CNN_Model_2",  # Model name          # Parameters for Logistic Regression
        model2,  # Model instance
        (X_train, y_train),     # Training data
        (X_test, y_test)        # Testing data
    ),
    (
        "CNN_Model_3",        # Model name
                     # Parameters for XGBClassifier
          model3,  # Model instance
        (X_train, y_train),     # Training data
        (X_test, y_test)        # Testing data
    ),

]


2025/11/24 22:00:32 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd0fce05de10148d0b9772617804e97d8', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/11/24 22:01:14 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'ccfd46ea1e384b78b7a7c7763e69e741', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


RandomForest accuracy = 0.9989


Epoch 1/5
1/5 ━━━━━━━━━━━━━━━━━━━━ 4s 1s/step - accuracy: 0.0312 - loss: 3.3372

5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 89ms/step - accuracy: 0.0849 - loss: 3.3110 - val_accuracy: 0.1348 - val_loss: 3.2798
Epoch 2/5
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1392 - loss: 3.2732

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step - accuracy: 0.1319 - loss: 3.2499 - val_accuracy: 0.1265 - val_loss: 3.2250
Epoch 3/5
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 38ms/step - accuracy: 0.1494 - loss: 3.2116

...

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.1141 - loss: 3.2534 - val_accuracy: 0.0886 - val_loss: 3.2452
Epoch 4/5
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step - accuracy: 0.1157 - loss: 3.2293

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.0962 - loss: 3.2167 - val_accuracy: 0.0665 - val_loss: 3.2122
Epoch 5/5
1/5 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.0767 - loss: 3.2013

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.0757 - loss: 3.1816 - val_accuracy: 0.0554 - val_loss: 3.1799
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


2025/11/24 22:01:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 973us/step
CNN_Model_3 accuracy = 0.0680


In [17]:
reports = []
for model_name,model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    if "Random Forest" in model_name:
         report = evaluate_and_log(model_name,lambda:model.predict(X_test))
    else:
         report = evaluate_and_log(
        model_name,
        lambda:np.argmax(model.predict(X_test), axis=1)

    )
    reports.append(report)

2025/11/24 22:01:41 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID 'd9ca7f7dbba045b3bd7b76e4da0c2feb', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current sklearn workflow
2025/11/24 22:02:17 INFO mlflow.utils.autologging_utils: Created MLflow autologging run with ID '835236ebd34248cc9ac3a4297048bf37', which will track hyperparameters, performance metrics, model artifacts, and lineage information for the current tensorflow workflow


Random Forest accuracy = 0.9989


321/339 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.3971 - loss: 2.1842

2025/11/24 22:02:18 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


339/339 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5765 - loss: 1.5607
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


...

304/339 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.1786 - loss: 2.8437

2025/11/24 22:02:29 WARNING mlflow.utils.checkpoint_utils: Checkpoint logging is skipped, because checkpoint 'save_best_only' config is True, it requires to compare the monitored metric value, but the provided monitored metric value is not available.


339/339 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.3090 - loss: 2.4931
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step


2025/11/24 22:02:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 898us/step
85/85 ━━━━━━━━━━━━━━━━━━━━ 0s 842us/step
CNN_Model_3 accuracy = 0.5473


In [18]:
reports

[{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 108.0},
  '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 81.0},
  '2': {'precision': 0.991304347826087,
   'recall': 1.0,
   'f1-score': 0.9956331877729258,
   'support': 114.0},
  '3': {'precision': 1.0,
   'recall': 0.9900990099009901,
   'f1-score': 0.9950248756218906,
   'support': 101.0},
  '4': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 70.0},
  '5': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 69.0},
  '6': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 108.0},
  '7': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 109.0},
  '8': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 108.0},
  '9': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 105.0},
  '10': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 108.0},
  '11': {'precision': 0.9904761904761905,
   'recall': 1.0,
   'f1-score': 0.9

In [19]:
def TFGP(model):
    architecture = {}
    c=0
    for layer in model.layers:
        architecture[f"layer_{c}"]=layer.get_config()
        c=c+1

    return architecture

In [20]:
mlflow.set_experiment("models_comparison")
mlflow.set_tracking_uri(uri="http://127.0.0.1:5000/")
example_input = X_test[:1]
for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    report = reports[i]
    signature = infer_signature(example_input, model.predict(example_input))
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_param("model", model_name) # Logs a single parameter with a specific name and value.
        if "Random Forest" in model_name:
            mlflow.log_params(model.get_params())
        else:
            mlflow.log_params(TFGP(model)) #  Logs multiple parameters at once from a dictionary.
        # mlflow.log_metrics({'accuracy':report['accuracy'],})
        mlflow.log_metric('accuracy', report['accuracy'])
        mlflow.log_metric('recall_class_1', report['1']['recall'])
        mlflow.log_metric('recall_class_0', report['0']['recall'])
        mlflow.log_metric('f1_score_macro', report['macro avg']['f1-score'])        
        
        if "Random Forest" in model_name:
            mlflow.sklearn.log_model(model, model_name,input_example=example_input, signature=signature)
        else:
            mlflow.tensorflow.log_model(model, model_name, input_example=example_input, signature=signature)  

2025/11/24 22:02:41 INFO mlflow.tracking.fluent: Experiment with name 'models_comparison' does not exist. Creating a new experiment.
2025/11/24 22:02:41 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/670819372575881117/runs/43d8812d80a147d6bca49d45f1f0336b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/670819372575881117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step


2025/11/24 22:02:46 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step
🏃 View run CNN_Model_2 at: http://127.0.0.1:5000/#/experiments/670819372575881117/runs/1b1eee48b31042e996eda30d40858ccc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/670819372575881117
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step


2025/11/24 22:02:58 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
🏃 View run CNN_Model_3 at: http://127.0.0.1:5000/#/experiments/670819372575881117/runs/144c67f724c442ecb2c58ccf0a413dde
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/670819372575881117


In [22]:
runs=["43d8812d80a147d6bca49d45f1f0336b","1b1eee48b31042e996eda30d40858ccc","144c67f724c442ecb2c58ccf0a413dde"]
for i, element in enumerate(models):
    model_name = element[0]
    model = element[1]
    run_id = runs[i]
    model_uri = f'runs:/{run_id}/{model_name}'
    print(model_uri)
    with mlflow.start_run(run_id=run_id):
     mlflow.register_model(model_uri=model_uri, name=model_name)

Successfully registered model 'Random Forest'.
2025/11/24 22:05:03 WARNING mlflow.tracking._model_registry.fluent: Run with id 43d8812d80a147d6bca49d45f1f0336b has no artifacts at artifact path 'Random Forest', registering model based on models:/m-c2145ef0eb55443d9a6e5658b6013751 instead
2025/11/24 22:05:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Random Forest, version 1


runs:/43d8812d80a147d6bca49d45f1f0336b/Random Forest


Created version '1' of model 'Random Forest'.
Successfully registered model 'CNN_Model_2'.
2025/11/24 22:05:03 WARNING mlflow.tracking._model_registry.fluent: Run with id 1b1eee48b31042e996eda30d40858ccc has no artifacts at artifact path 'CNN_Model_2', registering model based on models:/m-a948574eb8c642c7afbf9e82acd422e7 instead
2025/11/24 22:05:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CNN_Model_2, version 1


🏃 View run Random Forest at: http://127.0.0.1:5000/#/experiments/670819372575881117/runs/43d8812d80a147d6bca49d45f1f0336b
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/670819372575881117
runs:/1b1eee48b31042e996eda30d40858ccc/CNN_Model_2


Created version '1' of model 'CNN_Model_2'.
Successfully registered model 'CNN_Model_3'.
2025/11/24 22:05:03 WARNING mlflow.tracking._model_registry.fluent: Run with id 144c67f724c442ecb2c58ccf0a413dde has no artifacts at artifact path 'CNN_Model_3', registering model based on models:/m-71338ab907934d66b00929c572042934 instead
2025/11/24 22:05:03 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: CNN_Model_3, version 1


🏃 View run CNN_Model_2 at: http://127.0.0.1:5000/#/experiments/670819372575881117/runs/1b1eee48b31042e996eda30d40858ccc
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/670819372575881117
runs:/144c67f724c442ecb2c58ccf0a413dde/CNN_Model_3
🏃 View run CNN_Model_3 at: http://127.0.0.1:5000/#/experiments/670819372575881117/runs/144c67f724c442ecb2c58ccf0a413dde
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/670819372575881117


Created version '1' of model 'CNN_Model_3'.


In [23]:
model_versions = [1,1,1]
for i, element in enumerate(models):
    model_name = element[0]
    model_uri = f"models:/{model_name}/{model_versions[i]}"
    loaded_model = mlflow.pyfunc.load_model(model_uri)
    if "Random Forest" in model_name:
        print(evaluate_and_log(model_name,lambda:loaded_model.predict(X_test)))
    else:
        print(evaluate_and_log(model_name,lambda:np.argmax(loaded_model.predict(X_test),axis=1)))

Random Forest accuracy = 0.9989
{'0': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 108.0}, '1': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 81.0}, '2': {'precision': 0.991304347826087, 'recall': 1.0, 'f1-score': 0.9956331877729258, 'support': 114.0}, '3': {'precision': 1.0, 'recall': 0.9900990099009901, 'f1-score': 0.9950248756218906, 'support': 101.0}, '4': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 70.0}, '5': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 69.0}, '6': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 108.0}, '7': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 109.0}, '8': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 108.0}, '9': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 105.0}, '10': {'precision': 1.0, 'recall': 1.0, 'f1-score': 1.0, 'support': 108.0}, '11': {'precision': 0.9904761904761905, 'recall': 1.0, 'f1-score': 0.9952153110047847

In [25]:
current_model_uri = f"models:/{models[0][0]}@challenger"
loaded_model = mlflow.sklearn.load_model(current_model_uri)

production_model_name = "random_forest-prod"

client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Successfully registered model 'random_forest-prod'.
Copied version '1' of model 'Random Forest' to version '1' of model 'random_forest-prod'.


<ModelVersion: aliases=[], creation_timestamp=1764014783554, current_stage='None', deployment_job_state=<ModelVersionDeploymentJobState: current_task_name='', job_id='', job_state='DEPLOYMENT_JOB_CONNECTION_STATE_UNSPECIFIED', run_id='', run_state='DEPLOYMENT_JOB_RUN_STATE_UNSPECIFIED'>, description='', last_updated_timestamp=1764014783554, metrics=None, model_id=None, name='random_forest-prod', params=None, run_id='43d8812d80a147d6bca49d45f1f0336b', run_link='', source='models:/Random Forest/1', status='READY', status_message=None, tags={}, user_id='', version='1'>

In [26]:
model_version = model_versions[0]
prod_model_uri = f"models:/{models[0][0]}@champion"

loaded_model = mlflow.sklearn.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([26, 25, 22, 25])